In [26]:
# provides a simple API to write functional/acceptance tests using Selenium WebDriver
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
import time
import pandas as pd

In [50]:
def get_jobs(keyword, num_jobs, verbose, path, slep_time):
    
    '''Gathers jobs as a dataframe, scraped from Glassdoor'''
    
    #Initializing the webdriver
    options = webdriver.ChromeOptions()
    
    #Uncomment the line below if you'd like to scrape without a new Chrome window every time.
    #options.add_argument('headless')
    
    #Change the path to where chromedriver is in your home folder.
    driver = webdriver.Chrome(executable_path=path, options=options)
    driver.set_window_size(1120, 1000)
    url = "https://www.glassdoor.ca/Job/jobs.htm?suggestCount=0&suggestChosen=false&clickSource=searchBtn&typedKeyword="+keyword+"&sc.keyword="+keyword+"&locT=&locId=&jobType=all&fromAge=-1&minSalary=0"
    #url = 'https://www.glassdoor.ca/Job/jobs.htm?sc.keyword="' + keyword + '"&locT=C&locId=1147401&locKeyword=San%20Francisco,%20CA&jobType=all&fromAge=-1&minSalary=0&includeNoSalaryJobs=true&radius=100&cityId=-1&minRating=0.0&industryId=-1&sgocId=-1&seniorityType=all&companyId=-1&employerSizes=0&applicationType=0&remoteWorkType=0'
    driver.get(url) #website to script data 
    jobs = []

    while len(jobs) < num_jobs:  #If true, should be still looking for new jobs.

        #Let the page load. Change this number based on your internet speed.
        #Or, wait until the webpage is loaded, instead of hardcoding it.
        time.sleep(slep_time)

        #Test for the "Sign Up" prompt and get rid of it.
        try:
            driver.find_element_by_class_name("selected").click()
        except ElementClickInterceptedException:
            pass

        time.sleep(.1)

        try:
            driver.find_element_by_css_selector('[alt="Close"]').click()  #clicking to the X.
        except NoSuchElementException:
            pass

        
        #Going through each job in this page
        job_buttons = driver.find_elements_by_class_name("jl")  #jl for Job Listing. These are the buttons we're going to click.
        for job_button in job_buttons:  

            print("Progress: {}".format("" + str(len(jobs)) + "/" + str(num_jobs)))
            if len(jobs) >= num_jobs:
                break

            job_button.click()  #You might 
            time.sleep(1)
            collected_successfully = False
            
            while not collected_successfully:
                try:
                    company_name = driver.find_element_by_xpath('.//div[@class="employerName"]').text
                    location = driver.find_element_by_xpath('.//div[@class="location"]').text
                    job_title = driver.find_element_by_xpath('.//div[contains(@class, "title")]').text
                    
                    job_description = driver.find_element_by_xpath('.//div[@class="jobDescriptionContent desc"]').text
                    collected_successfully = True
                except:
                    time.sleep(5)

            try:
                salary_estimate = driver.find_element_by_xpath('.//span[@class="gray salary"]').text
            except NoSuchElementException:
                salary_estimate = -1 #You need to set a "not found value. It's important."
            try:
                rating = driver.find_element_by_xpath('.//span[@class="rating"]').text
            except NoSuchElementException:
                rating = -1 #You need to set a "not found value. It's important."

            #Printing for debugging
            if verbose:
                print("Job Title: {}".format(job_title))
                print("Salary Estimate: {}".format(salary_estimate))
                print("Job Description: {}".format(job_description[:500]))
                print("Rating: {}".format(rating))
                print("Company Name: {}".format(company_name))
                print("Location: {}".format(location))

            #Going to the Company tab...
            #clicking on this:
            #<div class="tab" data-tab-type="overview"><span>Company</span></div>
            try:
                driver.find_element_by_xpath('.//div[@class="tab" and @data-tab-type="overview"]').click()

                try:
                    #<div class="infoEntity">
                    #    <label>Headquarters</label>
                    #    <span class="value">San Francisco, CA</span>
                    #</div>
                    headquarters = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Headquarters"]//following-sibling::*').text
                except NoSuchElementException:
                    headquarters = -1

                try:
                    size = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Size"]//following-sibling::*').text
                except NoSuchElementException:
                    size = -1

                try:
                    founded = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Founded"]//following-sibling::*').text
                except NoSuchElementException:
                    founded = -1

                try:
                    type_of_ownership = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Type"]//following-sibling::*').text
                except NoSuchElementException:
                    type_of_ownership = -1

                try:
                    industry = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Industry"]//following-sibling::*').text
                except NoSuchElementException:
                    industry = -1

                try:
                    sector = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Sector"]//following-sibling::*').text
                except NoSuchElementException:
                    sector = -1

                try:
                    revenue = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Revenue"]//following-sibling::*').text
                except NoSuchElementException:
                    revenue = -1

                try:
                    competitors = driver.find_element_by_xpath('.//div[@class="infoEntity"]//label[text()="Competitors"]//following-sibling::*').text
                except NoSuchElementException:
                    competitors = -1
                    
            except NoSuchElementException:  #Rarely, some job postings do not have the "Company" tab.
                headquarters = -1
                size = -1
                founded = -1
                type_of_ownership = -1
                industry = -1
                sector = -1
                revenue = -1
                competitors = -1

                
            if verbose:
                print("Headquarters: {}".format(headquarters))
                print("Size: {}".format(size))
                print("Founded: {}".format(founded))
                print("Type of Ownership: {}".format(type_of_ownership))
                print("Industry: {}".format(industry))
                print("Sector: {}".format(sector))
                print("Revenue: {}".format(revenue))
                print("Competitors: {}".format(competitors))
                print("@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@@")

            jobs.append({"Job Title" : job_title,
            "Salary Estimate" : salary_estimate,
            "Job Description" : job_description,
            "Rating" : rating,
            "Company Name" : company_name,
            "Location" : location,
            "Headquarters" : headquarters,
            "Size" : size,
            "Founded" : founded,
            "Type of ownership" : type_of_ownership,
            "Industry" : industry,
            "Sector" : sector,
            "Revenue" : revenue,
            "Competitors" : competitors})
            #add job to jobs

        #Clicking on the "next page" button
        try:
            driver.find_element_by_xpath('.//li[@class="next"]//a').click()
        except NoSuchElementException:
            print("Scraping terminated before reaching target number of jobs. Needed {}, got {}.".format(num_jobs, len(jobs)))
            break

    return pd.DataFrame(jobs)  #This line converts the dictionary object into a pandas DataFrame.

In [51]:
path = "/Users/lizha/Documents/ds_project/chromedriver"
df= get_jobs('data analyst',30,False,path,15)

Progress: 0/30
Progress: 1/30
Progress: 2/30
Progress: 3/30
Progress: 4/30
Progress: 5/30
Progress: 6/30
Progress: 7/30
Progress: 8/30
Progress: 9/30
Progress: 10/30
Progress: 11/30
Progress: 12/30
Progress: 13/30
Progress: 14/30
Progress: 15/30
Progress: 16/30
Progress: 17/30
Progress: 18/30
Progress: 19/30
Progress: 20/30
Progress: 21/30
Progress: 22/30
Progress: 23/30
Progress: 24/30
Progress: 25/30
Progress: 26/30
Progress: 27/30
Progress: 28/30
Progress: 29/30


In [52]:
df

,Company Name,Competitors,Founded,Headquarters,Industry,Job Description,Job Title,Location,Rating,Revenue,Salary Estimate,Sector,Size,Type of ownership
0,Rakuten Kobo\n3.8,"Amazon, Barnes & Noble",2009,"Toronto, Canada",Media & Entertainment Retail Shops,"Job Description:\nDataAnalyst, Finance\nHere a...",Data Analyst,Toronto,3.8,$100 to $500 million (CAD),CA$49k-CA$66k (Glassdoor Est.),Retail,201 to 500 employees,Company - Private
1,Syniti\n3.8,"Collabera, Commvault, LTI",1996,"Hyannis, MA",IT Services,"Syniti, formerly BackOffice Associates, was fo...",Data Analyst,Moncton,3.8,$100 to $500 million (CAD),CA$49k-CA$66k (Glassdoor Est.),Information Technology,501 to 1000 employees,Company - Private
2,NCRI\n3.0,-1,2007,"Mississauga, Canada",Computer Hardware & Software,Who Are We?\n\nAt National Credit Recovery Inc...,Data Analyst,Mississauga,3.0,Unknown / Non-Applicable,CA$49k-CA$66k (Glassdoor Est.),Information Technology,501 to 1000 employees,Company - Private
3,HomeStars\n3.6,-1,2006,"Toronto, Canada",Enterprise Software & Network Solutions,"WHO WE ARE:\n\nHomeowners invest their time, m...",Product Data Analyst,Toronto,3.6,Unknown / Non-Applicable,CA$49k-CA$66k (Glassdoor Est.),Information Technology,51 to 200 employees,Subsidiary or Business Segment
4,Natural Partners Fullscript\n4.7,-1,2012,"Ottawa, Canada",Internet,Fullscript is currently seeking a Sales Data A...,Sales Data Analyst,Ottawa,4.7,Unknown / Non-Applicable,CA$49k-CA$66k (Glassdoor Est.),Information Technology,201 to 500 employees,Company - Private
5,Plenty of Fish\n4.5,-1,2003,"Vancouver, Canada",Internet,We are looking for a Senior Data Analyst to be...,Senior Data Analyst,Vancouver,4.5,Unknown / Non-Applicable,CA$49k-CA$66k (Glassdoor Est.),Information Technology,51 to 200 employees,Company - Public
6,Data Sciences Inc.\n5.0,-1,2013,"Vienna, VA",Biotech & Pharmaceuticals,Job Description\nWe're looking for someone to ...,Data Analyst,Montreal,5.0,Unknown / Non-Applicable,CA$49k-CA$66k (Glassdoor Est.),Biotech & Pharmaceuticals,1 to 50 employees,Company - Private
7,Kano/Apps\n4.0,-1,2008,"Victoria, Canada",-1,Kano is looking for a Data Analyst to help us ...,Junior Data Analyst,Victoria,4.0,Unknown / Non-Applicable,CA$49k-CA$66k (Glassdoor Est.),-1,1 to 50 employees,Company - Private
8,Intrigue Media\n4.8,-1,2007,"Guelph, Canada",Advertising & Marketing,"At Intrigue, our purpose is empowering leaders...",Data Analyst and Reporting Specialist,Guelph,4.8,$1 to $5 million (CAD),CA$49k-CA$66k (Glassdoor Est.),Business Services,1 to 50 employees,Company - Private
9,AltisHR\n4.0,-1,1989,"Ottawa, Canada",Staffing & Outsourcing,Our client in the risk and compliance industry...,Entry level Data Analyst Contract,Vancouver,4.0,$100 to $500 million (CAD),CA$49k-CA$66k (Glassdoor Est.),Business Services,51 to 200 employees,Company - Private
